## Sternberg remapping of columns

The purpose of the remapping is to provide a finished event file `_events_temp4.tsv`
by remapping the corrected BIDS events files `_events_temp2.tsv'.

1. Run remapping of `value` to `event_type`, `task_role`, and `letter`.
2. Set the `trial` column based on the `ready` value in the `task_role`.
3. Count the number of targets in each trial and set the `memory_cond`.
4. Change the following `task_role` items based on contents:
  1. `remembered_correct` or `ignored_correct` to `continue` if last item in trial.
  2. `remembered_probe` to `ignored_probe` for probes corresponding to letters ignored in that trial.
  3. `remembered_probe` to `not_in_trial_probe` for probes corresponding to letters not shown in that trial.
  4. `remembered_correct` to `remembered_incorrect` to indicate right button but should have been left.
  5. `ignored_correct` to `ignored_incorrect` to indicate left button but should have been right.
  6. `bad_trial` if number of events in the trial is not 14 or 13. (Also check that if 13,
it is the feedback event that is missing.

In [1]:
from hed.tools.io_util import get_file_list
col_order = ['onset', 'duration', 'sample', 'event_type', 'task_role', 'letter', 'trial', 'memory_cond', 'value']
bids_root_path = 'G:/Sternberg/SternbergWorking/'
bids_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events_temp2")

In [2]:
from hed.tools.key_map import KeyMap
template_path = '../../../data/sternberg_data/sternberg_map.tsv'
key_cols = ['value']
target_cols = ['event_type', 'task_role', 'letter']
key_map = KeyMap(key_cols, target_cols, name='sternberg_remap')
duplicates = key_map.update(template_path)
if duplicates:
    print(f"Warning: this mapping has {len(duplicates)} duplicate key entries")

In [3]:
def set_trials(df):
    # Set the trials column based on start of trial marker value==ready
    df['trial'] = 'n/a'
    trial_num = 0
    for index, value in df['event_type'].iteritems():
        if value == 'show_cross':
            trial_num += 1
        df.loc[index, 'trial'] = trial_num

def set_memory_cond(df):
    # Set the memory_cond to number of targets in the trial
    df['memory_cond'] = 0
    max_trial = df['trial'].max()
    target_mask = df['task_role'].map(str) == 'target'
    for index in range(1, max_trial+1):
        trial_mask = (df['trial'] == index)
        df.loc[trial_mask, 'memory_cond'] =  sum(trial_mask & target_mask)

def set_probe(df):
    # Set the task_role=probe_nontarget if task_role==probe_target and the letter non-target
    max_trial = df['trial'].max()
    probe_mask = df['task_role'] == 'probe_target'
    target_mask = df['task_role'] == 'target'
    nontarget_mask = df['task_role'] == 'nontarget'
    for index in range(1, max_trial+1):
        trial_mask = (df['trial'] == index)
        target_set = set(df.loc[trial_mask & target_mask, 'letter'])
        nontarget_set = set(df.loc[trial_mask & nontarget_mask, 'letter'])
        next_mask = trial_mask & probe_mask
        if not sum(next_mask):
            print(f"...warning trial {index} does not have a probe letter")
            continue
        probe_letter = df.loc[next_mask, 'letter'].values[0]
        if probe_letter in target_set:
            continue
        elif probe_letter in nontarget_set:
            df.loc[trial_mask & probe_mask, 'task_role'] = 'probe_nontarget'
        else:
            df.loc[trial_mask & probe_mask, 'task_role'] = 'probe_out'

def set_continue(df):
    # Set the last event in each trial to have task_role=continue
    max_trial = df['trial'].max()
    for index in range(1, max_trial+1):
        df_col = df.loc[df['trial'] == index, 'task_role']
        index = df_col.index.values
        df.loc[index[-1], 'task_role'] = 'continue'

def set_response(df):
    # Set the task_role=in_group_correct if task_role==probe_target and the letter non-target
    max_trial = df['trial'].max()
    probe_in_mask = df['task_role'] == 'probe_target'
    probe_out_mask = (df['task_role'] == 'probe_nontarget') | (df['task_role'] == 'probe_out')
    right_mask = (df['event_type'] == 'right_click') & (df['task_role'] == 'in_group_correct')
    left_mask = (df['event_type'] == 'left_click') & (df['task_role'] == 'out_group_correct')
    for index in range(1, max_trial+1):
        trial_mask = (df['trial'] == index)
        out_group_incorrect = trial_mask & probe_in_mask & left_mask
        in_group_incorrect = trial_mask & probe_out_mask & right_mask

        if sum(out_group_incorrect):
            df.loc[out_group_incorrect, 'task_role'] = 'out_group_incorrect'
        elif sum(in_group_incorrect):
            df.loc[in_group_incorrect, 'task_role'] = 'in_group_incorrect'

def set_bad_trials(df):
    # Set the task_role=in_group_correct if task_role==probe_target and the letter non-target
    max_trial = df['trial'].max()
    feedback_mask = (df['event_type'] == 'sound_beep') | (df['event_type'] == 'sound_buzz')
    for index in range(1, max_trial+1):
        trial_mask = df['trial'] == index
        if sum(trial_mask) == 14:
            continue
        elif sum(trial_mask) == 13 and not sum(feedback_mask & trial_mask):
            print(f"Warning: trial {index} has no feedback event")
        else:
            df.loc[df['trial'] == index, 'task_role'] = 'bad_trial'
            print(f"Trial {index} has {sum(df['trial'] == index)}")

for file in bids_files:
    print(f"Remapping {file}...")

    ## Remap the columns
    df_new, missing = key_map.remap(file)
    if missing:
        print(f"The keys {str(missing)} were not in the map for {file}")

    # Set the trials and  memory_cond columns
    set_trials(df_new)
    set_memory_cond(df_new)

    # Correct the probe column values to account for non-target or missing probes
    set_probe(df_new)

    # Set task_role as continue for last click in trial
    set_continue(df_new)

    # Set task_role to indicate the correct response
    set_response(df_new)

    # Set bad trails if trial does not have 14 events

    set_bad_trials(df_new)

    # Save the new events file
    df_new = df_new.reindex(columns=col_order)
    filename = file[:-5] + "4.tsv"
    print(f"...Writing remapped file to {filename}")
    df_new.to_csv(filename, sep='\t', index=False)


Remapping G:/Sternberg/SternbergWorking/sub-001\ses-01\eeg\sub-001_ses-01_task-Experiment_run-1_events_temp2.tsv...
...Writing remapped file to G:/Sternberg/SternbergWorking/sub-001\ses-01\eeg\sub-001_ses-01_task-Experiment_run-1_events_temp4.tsv
Remapping G:/Sternberg/SternbergWorking/sub-001\ses-01\eeg\sub-001_ses-01_task-Experiment_run-2_events_temp2.tsv...
...Writing remapped file to G:/Sternberg/SternbergWorking/sub-001\ses-01\eeg\sub-001_ses-01_task-Experiment_run-2_events_temp4.tsv
Remapping G:/Sternberg/SternbergWorking/sub-001\ses-01\eeg\sub-001_ses-01_task-Experiment_run-3_events_temp2.tsv...
...Writing remapped file to G:/Sternberg/SternbergWorking/sub-001\ses-01\eeg\sub-001_ses-01_task-Experiment_run-3_events_temp4.tsv
Remapping G:/Sternberg/SternbergWorking/sub-001\ses-01\eeg\sub-001_ses-01_task-Experiment_run-4_events_temp2.tsv...
...Writing remapped file to G:/Sternberg/SternbergWorking/sub-001\ses-01\eeg\sub-001_ses-01_task-Experiment_run-4_events_temp4.tsv
Remapping G:

In [4]:
from hed.tools.io_util import make_file_dict
from hed.tools.map_util import make_combined_dicts

event_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="_events_temp4")
file_dict = make_file_dict(event_files, [0, -3])
print('\nRefactored events summary:')
skip_cols = ['onset', 'duration', 'sample']
bids_dicts_all, bids_dicts =  make_combined_dicts(file_dict, skip_cols=skip_cols)
bids_dicts_all.print()


Refactored events summary:
Summary for column dictionary :
  Categorical columns (6):
    event_type (7 distinct values):
      left_click: 1828
      right_click: 2382
      show_cross: 2106
      show_dash: 2097
      show_letter: 18880
      sound_beep: 1868
      sound_buzz: 128
    letter (24 distinct values):
      +: 2106
      -: 2097
      B: 826
      C: 1026
      D: 918
      F: 836
      G: 984
      H: 964
      J: 889
      K: 937
      L: 883
      M: 1122
      N: 716
      P: 831
      Q: 913
      R: 1075
      S: 883
      T: 833
      V: 856
      W: 819
      X: 751
      Y: 752
      Z: 1066
      n/a: 6206
    memory_cond (4 distinct values):
      0: 32
      3: 9713
      5: 9820
      7: 9724
    task_role (12 distinct values):
      bad_trial: 69
      continue: 2095
      feedback_correct: 1866
      feedback_incorrect: 119
      fixate: 2095
      in_group_correct: 1109
      nontarget: 6283
      out_group_correct: 986
      probe_out: 933
      probe_ta

In [5]:
for key, edict in bids_dicts.items():
    trials = edict.categorical_info['trial']
    if '0' in trials.keys():
        print(f"{key}: {trials}")